Running a model doing sentiment analysis. Taking the direct model invocation approach.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers.pipelines import SUPPORTED_TASKS

model_name = 'cardiffnlp/twitter-roberta-base-sentiment-latest'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

print(model.config.id2label)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /Users/afecioru/.cache/huggingface/hub/models--cardiffnlp--twitter-roberta-base-sentiment-latest/snapshots/3216a57f2a0d9c45a2e6c20157c20c49fb4bf9c7/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "dtype": "float32",
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 

{0: 'negative', 1: 'neutral', 2: 'positive'}


Attempting to create safetensors variant
Safetensors PR exists


In [3]:
input_texts = [
  'I loved the movie, it was fantastic!',
  'This movie was terrible and boring.',
  'Best film ever!',
  'Worst experience of my life.',
]

In [ ]:
for input_text in input_texts:
  tokens = tokenizer(input_text, return_tensors='pt')
  # print(tokens)

  model_outputs = model(**tokens)
  # print(model_outputs)

  predicted_label = torch.argmax(model_outputs.logits).item()
  probs = torch.nn.functional.softmax(model_outputs.logits, dim=-1)
  sentiment = model.config.id2label[predicted_label]
  print(f'Text: {input_text}')
  print(f'Logits: {model_outputs.logits[0].tolist()}')
  print(f'Predicted label: {predicted_label}')
  print(f'Probabilities: {probs[0].tolist()}')
  print(f'Predicted sentiment: {sentiment}')
  print('----------------------------------------')

The same can be achieved using the `pipeline` high-lvl APIs. 

In [8]:
sentiment = pipeline('sentiment-analysis', model = model_name)

for text in input_texts:
  result = sentiment(text)
  print(result)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


[{'label': 'positive', 'score': 0.9880495667457581}]
[{'label': 'negative', 'score': 0.9440518617630005}]
[{'label': 'positive', 'score': 0.9761729836463928}]
[{'label': 'negative', 'score': 0.9420945048332214}]


In [ ]:
# list the names of supported tasks
for i, task in enumerate(sorted(SUPPORTED_TASKS.keys()), 1):
  print(f'{i:2}. {task}')